In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import ARCH, GARCH

from utilsforecast.losses import *
from utilsforecast.evaluation import evaluate

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

from mlforecast import MLForecast

warnings.filterwarnings("ignore")
os.environ["NIXTLA_ID_AS_COL"] = "true"
pd.set_option('display.precision', 3)

In [ ]:
plt.rcParams['figure.figsize'] = (9,6)

In [ ]:
url = "https://raw.githubusercontent.com/marcopeix/AppliedTimeSeriesForecastingInPython/refs/heads/master/data/AMZN.csv"
df = pd.read_csv(url, parse_dates=['Date'])
df.insert(0, 'unique_id', 1)

df = df[['unique_id', 'Date', 'Adj Close']]
df = df.rename(columns={'Adj Close': 'y', 'Date': 'ds'})

df.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'], df['y'])
ax.set_xlabel('Date')
ax.set_ylabel('Closing price (USD)')
ax.set_title('Daily closing price of AMZN')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
# Get percentage change

df.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'], df['return'])
ax.set_xlabel('Date')
ax.set_ylabel('Percentage return (%)')
ax.set_title('Daily percentage return of AMZN')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
plot_acf(df['return']);

In [ ]:
# Get square of return

df.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'], df['sq_return'])
ax.set_xlabel('Date')
ax.set_ylabel('Percentage return (%)')
ax.set_title('Daily percentage return of AMZN')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
plot_pacf(df['sq_return']);

## ARCH

In [ ]:
# Define different ARCH models

In [ ]:
df_fit = df[['unique_id', 'ds', 'return']]
df_fit.head()

In [ ]:
h = 5

# Initialize StatsForecast with ARCH models

cv_df = sf.cross_validation(h=h, 
                            df=df_fit, 
                            n_windows=10, 
                            step_size=h,
                            target_col='return',
                            refit=True)

cv_df.head()

In [ ]:
cv_df = cv_df.merge(df[['ds', 'y']], on='ds', how='left')
cv_df.head()

In [ ]:
# Get initial value to inverse transform the percentage change

initial_value

In [ ]:
cv_df['ARCH(1)'] = initial_value * (1+cv_df['ARCH(1)']).cumprod()
cv_df['ARCH(2)'] = initial_value * (1+cv_df['ARCH(2)']).cumprod()
cv_df['ARCH(3)'] = initial_value * (1+cv_df['ARCH(3)']).cumprod()
cv_df.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'], df['y'])
ax.plot(cv_df['ds'], cv_df['ARCH(1)'], label='ARCH(1)', ls='--')
ax.plot(cv_df['ds'], cv_df['ARCH(2)'], label='ARCH(2)', ls=':')
ax.plot(cv_df['ds'], cv_df['ARCH(3)'], label='ARCH(3)', ls='-.')
ax.set_xlabel('Date')
ax.set_ylabel('Closing price (USD)')
ax.set_title('Daily closing price of AMZN')
ax.legend(loc='best')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
eval_df = cv_df.drop(['ds', 'cutoff', 'return'], axis=1)
evaluation = evaluate(df=eval_df, metrics=[mae])
evaluation

## GARCH

In [ ]:
# Initialize different GARCH models

In [ ]:
sf = StatsForecast(models=[garch1, garch2, garch3], freq='B')

cv_df = sf.cross_validation(h=h, 
                            df=df_fit, 
                            n_windows=10, 
                            step_size=h,
                            target_col='return',
                            refit=True)

cv_df.head()

In [ ]:
cv_df = cv_df.merge(df[['ds', 'y']], on='ds', how='left')
cv_df['GARCH(1,1)'] = initial_value * (1+cv_df['GARCH(1,1)']).cumprod()
cv_df['GARCH(2,2)'] = initial_value * (1+cv_df['GARCH(2,2)']).cumprod()
cv_df['GARCH(3,3)'] = initial_value * (1+cv_df['GARCH(3,3)']).cumprod()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'], df['y'])
ax.plot(cv_df['ds'], cv_df['GARCH(1,1)'], label='GARCH(1,1)', ls='--')
ax.plot(cv_df['ds'], cv_df['GARCH(2,2)'], label='GARCH(2,2)', ls=':')
ax.plot(cv_df['ds'], cv_df['GARCH(3,3)'], label='GARCH(3,3)', ls='-.')
ax.set_xlabel('Date')
ax.set_ylabel('Closing price (USD)')
ax.set_title('Daily closing price of AMZN')
ax.legend(loc='best')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
eval_df = cv_df.drop(['ds', 'cutoff', 'return'], axis=1)
evaluation = evaluate(df=eval_df, metrics=[mae])
evaluation

## Forecasting with ML models and features

In [ ]:
data_url = 'https://raw.githubusercontent.com/panambY/Hourly_Energy_Consumption/master/data/PJM_Load_hourly.csv'
df = pd.read_csv(data_url, parse_dates=['Datetime'])
df.columns = ['ds', 'y']
df.insert(0, 'unique_id', 'PJM')
df['ds'] = pd.to_datetime(df['ds'])
df = df.sort_values(['unique_id', 'ds']).reset_index(drop=True)
df.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'][-700:], df['y'][-700:])
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
threshold_time = df['ds'].max() - pd.Timedelta(hours=24)

df_train = df[df['ds'] <= threshold_time]
df_test = df[df['ds'] > threshold_time]

print(len(df_train), len(df_test))

## Analyzing seasonality and frequencies

In [ ]:
from scipy.fftpack import fft, fftfreq

def analyze_frequencies(signal, sampling_rate):
    
    # Perform FFT
    
    # Get frequencies corresponding to FFT values
    # For hourly data, this gives us frequencies in cycles per day
    
    # Get positive frequencies only (up to Nyquist frequency)
    
    # Find indices of top 5 amplitudes
    
    # Get top frequencies and their amplitudes
    
    return top_frequencies, top_amplitudes

In [ ]:
top_frequencies, top_amplitudes = analyze_frequencies(df_train['y'], sampling_rate=24)

In [ ]:
top_frequencies

In [ ]:
fig, ax = plt.subplots()
    
# Create bar plot
bars = ax.bar(top_frequencies, top_amplitudes, 
                color='skyblue',
                edgecolor='navy',
                width=0.1)  # Adjust width as needed

# Add value labels on top of each bar
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}',
            ha='center',
            va='bottom',
            fontweight='bold')

# Customize the plot
ax.set_xlabel('Frequency (cycles/day)', fontsize=12)
ax.set_ylabel('Amplitude', fontsize=12)

# Add grid
ax.grid(True, linestyle='--', alpha=0.7)

# Adjust layout
plt.tight_layout()

## Model selection

In [ ]:
import lightgbm as lgb
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

from mlforecast.lag_transforms import ExpandingMean, RollingMean
from mlforecast.target_transforms import Differences


In [ ]:
# Initialize model dict
models ={}

In [ ]:
# Initialize MLForecast object


In [ ]:
ml_cv_df = mlf.cross_validation(
    df=df_train,
    h=24,
    n_windows=7,
    step_size=24,
    refit=False,
)
ml_cv_df.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(ml_cv_df['ds'], ml_cv_df['y'])
ax.plot(ml_cv_df['ds'], ml_cv_df['lgbm'], label='LightGBM', ls='--')
ax.plot(ml_cv_df['ds'], ml_cv_df['lasso'], label='Lasso', ls=':')
ax.plot(ml_cv_df['ds'], ml_cv_df['gbr'], label='GradientBoosting', ls='-.')
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
eval_df = ml_cv_df.drop(['ds', 'cutoff'], axis=1)
evaluation = evaluate(df=eval_df, metrics=[mae, smape])
evaluation

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import MSTL

In [ ]:
mstl = MSTL(season_length=[12, 24, 168],
            alias='MSTL')

sf = StatsForecast(models=[mstl], freq='H')
mstl_cv_df = sf.cross_validation(
    h=24, 
    df=df_train, 
    n_windows=7,
    step_size=24
)

eval_df = mstl_cv_df.drop(['ds', 'cutoff'], axis=1)
evaluation = evaluate(df=eval_df, metrics=[mae, smape])
evaluation

## Fit and predict with the best model

In [ ]:
from mlforecast.utils import PredictionIntervals

In [ ]:
# Select best models to fit and predict

mlf = MLForecast(
    models = models, 
    freq='H',
    target_transforms=[Differences([24, 168])],
    lags=[1,12,24,168],
    lag_transforms={  
        1: [ExpandingMean()],
        24: [RollingMean(window_size=24)],
    },
    date_features=['month', 'hour', 'dayofweek']
)

mlf.fit(
    df = df_train,
    prediction_intervals=PredictionIntervals(n_windows=4, h=24)
)

levels = [80] 
preds = mlf.predict(24, level=levels)
preds.head()


In [ ]:
preds = preds.merge(df_test[['ds', 'y']], on='ds', how='left')

eval_df = preds.drop(['ds'], axis=1)
evaluation = evaluate(df=eval_df, metrics=[mae, smape], models=['lgbm', 'knn'])
evaluation

## Analyzing our ML model

In [ ]:
import shap



In [ ]:
shap.plots.bar(shap_values, show=False);

### Explain a single prediction

In [ ]:
from mlforecast.callbacks import SaveFeatures